# Instalando bibliotecas

In [ ]:
""" Installing libraries """

# %pip install --quiet pandas==2.3.2 matplotlib==3.10.6 seaborn==0.13.2 scikit-learn==1.7.1 numpy==2.2.6 pyarrow==21.0.0
# %pip install --quiet torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu129

# Importando bibliotecas (externas e próprias)

In [ ]:
""" Importing libraries """

# Add the parent directory to sys.path so 'Modules' can be imported

# Our modules
from Modules.loading.plug_n_play import get_clean_data
from Modules.hyperparams import get_hyperparams
from Modules.models.predictions import get_dataframe_predictions
from Modules.evaluation.output_formatter import forecast_to_output

# Importação do Dataset

É interessante dividir o treino em batches (mini-conjuntos de treino). Cada batch possui o tamanho de input size, seguindo a ordem cronológica de vendas dentro daquela janela de dias. No entanto, durante o treinamento é **ESSENCIAL** que a escolha do próximo batch seja aleatória.

Ex.: Inicia o treino por 21-27 jul e prevê 28, depois pula para 02-08 fev para prever 03. Esse processo deve ser repetido até todos os dados serem treinados, finalizando **01 epoch**.

O número de **epochs** diz o número total de iterações do modelo com relação ao dataset inteiro.

Sobre a composição da janela de input dentro de um batch, existem duas abordagens:

1) Treinar em cada janela todas as séries (pense que cada par produto-loja x tempo representa uma série temporal dentro daquele período). Esse modelo é bem mais complexo pois o output deve ter o mesmo tamanho de produto-loja.
2) Treinar vários modelos separados (considerando uma série temporal para cada modelo). Esse método é ineficiente pois o modelo nunca irá aprender os padrões entre as séries.
3) Treinar o modelo com um par produto-loja por vez. Ou seja:
   - O modelo realiza epochs = N iterações de treino ao longo de todo dataset
     - Em cada epoch, passa por todas as M batches
       - Em cada batch (que possui uma janela de tamanho input_size), atualiza os parâmetros para cada série temporal ($x_l,y_l$). Totalizando L atualizações, com L sendo o número de pares produto-loja.

Ressalta-se que cada conjunto ($x_l,y_l$) representa:
- $x_l$: série temporal do l-ésimo par produto-loja, sendo um vetor de tamanho input_size x (features + 1)
- $y_l$: Previsão de vendas do l-ésimo par produto-loja para os próximos $output\_size$ dias


In [ ]:
HYPERPARAMS = get_hyperparams() # Defining hyper-parameters
clean_data = get_clean_data(HYPERPARAMS) # Recebe os dados filtrados / limpos
final_predictions = get_dataframe_predictions(clean_data, HYPERPARAMS) # Modularizando o código de predição
final_df = forecast_to_output(final_predictions) # Output Formatting